### Missing Not At Random

In [1]:
import os
os.chdir("..")
import numpy as np
import pandas as pd

In [2]:
from utils.data import create_adult_dataset

In [3]:
data = create_adult_dataset()
X = data.X.copy().drop(columns=data.protected_features)
X.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,hours-per-week
0,39,7,9,13,4,1,1,4,40
1,50,6,9,13,2,4,0,4,13
2,38,4,11,9,0,6,1,4,40
3,53,4,1,7,2,6,0,2,40
4,28,4,9,13,2,10,5,2,40


In [4]:
X.shape

(32561, 9)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

def convert_multiple_features(K_df, range_min, range_max, scalar_0=0.05, scalar_1=2):
    max_iter = 10
    while(max_iter > 0):
        scalars = np.random.standard_normal(K_df.shape) * scalar_0
        scalar_a = np.random.standard_normal() - scalar_1
        K = K_df.to_numpy()
        K = scaler.fit_transform(K)
        M = scalar_a + (scalars * K)
        p = 1 / (1 + np.exp(-M))
        ratio = p.mean(axis=0)
        if ratio.min() > range_min and ratio.max() < range_max:
            break
        max_iter -= 1
    if max_iter <= 0:
        return None
    p_bi = np.random.binomial(1, p).astype(np.bool)
    K_df = K_df.where(~p_bi, other=np.nan)
    return K_df

In [6]:
ls = []
for _ in range(100):
    K_df_converted = convert_multiple_features(X, 0.1, 0.3)
    ls.append(K_df_converted.isnull().sum().mean() / X.shape[0])
ls

[0.1877979450535576,
 0.15747537101303877,
 0.1086234725250726,
 0.2260372838672031,
 0.2550665588348706,
 0.2496067210603005,
 0.14375070380721314,
 0.12310910462072897,
 0.1046855645301639,
 0.12899549222143739,
 0.17289941272619935,
 0.20687325327846195,
 0.1288487590812458,
 0.14448095710956188,
 0.13437343242938893,
 0.12952782640445795,
 0.18056366000225219,
 0.12699582663650108,
 0.14648744749171638,
 0.10699917078713798,
 0.10416687994157973,
 0.12209562223382438,
 0.24047855478094107,
 0.21274599128473395,
 0.12010278144610628,
 0.12439557889636203,
 0.2752201850202526,
 0.20297970646547167,
 0.23796702940463882,
 0.12173049558264999,
 0.14645332350562534,
 0.18172728792795745,
 0.237909018628284,
 0.20649447703285118,
 0.13501837576651002,
 0.16792072315551326,
 0.21165061133121083,
 0.19185869939839414,
 0.10356288538776792,
 0.10993042119236032,
 0.21658493971997858,
 0.18961675351221127,
 0.25152448907861824,
 0.1429351405396367,
 0.13167422512958585,
 0.12445700207132596,